### Loocv

In [65]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
df = sns.load_dataset('tips')


In [5]:
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [7]:
# total_billからtipを予測する
# データ準備
X = df['total_bill'].values.reshape(-1, 1)
y = df['tip']



In [12]:
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()

In [13]:
loo.split(X)

<generator object BaseCrossValidator.split at 0xffff3fda18b0>

In [16]:
list(loo.split(X))[:2]

[(array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
          14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
          27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
          40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
          53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
          66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
          79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
          92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
         105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
         118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
         131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
         144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
         157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
         170, 171, 172, 173, 174, 175,

In [21]:
from sklearn.metrics import mean_squared_error
mse_list = []
model = LinearRegression()
for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # モデル学習
    model.fit(X_train, y_train)
    # テストデータの予測
    y_pred = model.predict(X_test)
    # 評価

    mse = mean_squared_error(y_test, y_pred)
    mse_list.append(mse)



In [24]:
print(f'MSE(LOOCV) :{np.mean(mse_list)}')

MSE(LOOCV) :1.0675673489857438


### Cross Validation

In [31]:
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
cv = LeaveOneOut()
scores = cross_val_score(model, X, y, cv=cv, scoring = 'neg_mean_squared_error')
print(f'MSE(LOOCV) : {-np.mean(scores)}')


MSE(LOOCV) : 1.0675673489857438



### K-Fold CV

In [36]:
from sklearn.model_selection import KFold
mse_list = []
model = LinearRegression()
k = 5
cv = KFold(n_splits=k, shuffle=True, random_state=0)
# list(cv.split(X))[:2]
for train_index, test_index in cv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # 標準化できる

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    mse_list.append(mse)
    

In [39]:
print(f'MSW(k-fold) : {np.mean(mse_list)}')

MSW(k-fold) : 1.0802110883943916


In [40]:
k = 5
cv = KFold(n_splits=k, shuffle=True, random_state=0)
scores = cross_val_score(model, X, y, cv=cv, scoring='neg_mean_squared_error', n_jobs=-1)


In [41]:
scores

array([-0.82130906, -1.07458421, -1.08801239, -1.33238677, -1.084763  ])

In [44]:
print(f'K-fold MSE : {-np.mean(scores)}')

K-fold MSE : 1.0802110883943916


### K-Foldをrandom_stateで固定せずにやると、評価が変わってしまう→だから、複数回k-foldやって平均を取る手法
### Repeated k-fold CVを使う

In [49]:
from sklearn.model_selection import RepeatedKFold
k = 5
n_repeats = 3
cv = RepeatedKFold(n_splits=k, n_repeats=n_repeats, random_state=0)
scores= cross_val_score(model, X, y, cv=cv, scoring='neg_mean_squared_error')
scores # 三回あるから 3 x 5 = 15になる

array([-0.82130906, -1.07458421, -1.08801239, -1.33238677, -1.084763  ,
       -1.15878391, -1.6042084 , -1.03070862, -0.71202907, -0.84729854,
       -0.88561033, -1.52485216, -0.6332659 , -1.2003542 , -1.12141427])

In [50]:
print(f'k-fold MSE : {-np.mean(scores)}')

k-fold MSE : 1.0746387233165982


### CV時の特徴量スケーリング

### Pipelne

In [62]:
# Pipeline + Fold

In [57]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
pipeline = Pipeline(steps=[('scaler', StandardScaler()), ('model', LinearRegression())])

In [58]:
pipeline

Pipeline(steps=[('scaler', StandardScaler()), ('model', LinearRegression())])

In [60]:
cv = KFold(n_splits=5, shuffle=True, random_state=0)
scores = cross_val_score(pipeline, X, y, cv=cv, scoring='neg_mean_squared_error') # modelの場所にpipelineを入れた


In [61]:
scores


array([-0.82130906, -1.07458421, -1.08801239, -1.33238677, -1.084763  ])

### Pipelineなし + 標準化

In [72]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

model = LinearRegression()
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)

In [73]:
y_pred

array([2.71486884, 2.78639251, 2.90900452, 1.65836207, 2.57999564,
       1.50509707, 2.74858715, 3.30136293, 2.77208778, 4.45800284,
       3.50060744, 3.49345507, 2.35520697, 2.24587793, 2.28879213,
       4.02375199, 1.77075641, 2.3480546 , 2.83645908, 3.2778623 ,
       3.98901192, 3.05511716, 2.55240794, 2.45431834, 2.29798803,
       2.59327861, 2.16004953, 3.96244599, 3.50162921, 2.5289073 ,
       2.42264357, 2.19274606, 2.49314547, 1.99963215, 2.78639251,
       2.28572683, 2.64743224, 1.97306622, 5.85577969, 2.55036441,
       1.79425705, 2.18763723, 2.52073317, 3.96755482, 2.22135553,
       2.65151931, 2.78128368, 3.12255376, 2.66173698, 3.66409011,
       4.2567148 , 2.74552185, 3.01118119, 5.83943142, 1.89847725,
       2.14676656, 3.97572896, 3.03161652, 2.37462053, 2.21113786,
       3.70496078, 2.53299437, 3.07963956, 3.47199797, 3.99718606,
       2.5043849 , 2.60043097, 4.2720413 , 1.97306622, 3.87763935,
       2.4890584 , 1.99145802, 3.43010554, 2.37972937])

### Pipeline + 標準化

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
pipeline = Pipeline(steps=[('scaler', StandardScaler()), ('model', LinearRegression())])
pipeline